In [63]:
import re
import cv2
import json
from tqdm.notebook import tqdm
import csv
import shutil
import os
import glob
from natsort import natsorted as natsort
from random import shuffle
import random
import pandas as pd

## Download handwrite Data and Processing

※ (file_path) & (output_path) 以實際檔案位置需求作修改、替換，解壓縮後資料夾名稱為 cleaned_data，共684,677個圖片

git clone https://github.com/chenkenanalytic/handwritting_data_all.git

cat (file_path)/all_data.zip* > (file_path)/all_data.zip

unzip -O big5 (file_path)/all_data.zip -d (output_path)

In [36]:
handwrite_root_path = '/home/tingwei/signboard/test' ## The path we define in the above cell (output_path)

In [24]:
train_path = '/home/tingwei/signboard/test/train' ## The path of train data
darknet_path = '/home/tingwei/signboard/test/darknet' ## The root path of YoloV4(darkenet)

test_path = '/home/tingwei/signboard/private/img_private' ## The path of test data

In [26]:
if not os.path.exists(os.path.join(train_path, 'merge_training')):
    os.mkdir(os.path.join(train_path, 'merge_training'))
    print('make direction')

make direction


In [27]:
if not os.path.exists(os.path.join(train_path, 'only_text')):
    os.mkdir(os.path.join(train_path, 'only_text'))
    print('make direction')

make direction


In [40]:
# ## make the label.csv and rename in the full handwriting dataset

hand_write_path = os.path.join(handwrite_root_path, 'cleaned_data')
hand_write_list = natsort(glob.glob(os.path.join(hand_write_path, '*', '*')))

from tqdm.notebook import tqdm

first_index = 0
tmp_label = '一'

for i in tqdm(range(len(hand_write_list))):
    label = os.path.split(hand_write_list[i])[-1].split('_')[0]
    index = os.path.split(hand_write_list[i])[-1].split('_')[1].split('.')[0]
    folder_path = os.path.split(hand_write_list[i])[0]

    os.rename(hand_write_list[i], os.path.join(folder_path, str(first_index) + '_' + index + '.png'))

    with open(os.path.join(train_path, 'merge_training', 'hand_write_text_label_all.csv'), 'a+', newline='') as csvfile:
        # 建立 CSV 檔寫入器
        writer = csv.writer(csvfile)
        writer.writerow([os.path.join(folder_path, str(first_index) + '_' + index + '.png'), label])
    if label == tmp_label:
        pass
    else:
        tmp_label = label
        first_index += 1

## Training and Test Data Processing

In [41]:
json_path = os.path.join(train_path, 'json')
json_list = natsort(os.listdir(json_path))

In [42]:
def is_all_chinese(strs):
    for _char in strs:
        if not '\u4e00' <= _char <= '\u9fa5':
            return False
    return True

In [43]:

all_label = []
all_points = []
all_names = []
all_width = []
all_height = []
# 開啟 JSON 檔案
for i in tqdm(range(len(json_list))):
    with open(os.path.join(json_path, json_list[i])) as f:
        img_label = []
        img_points = []

        # 讀取 JSON 檔案
        p = json.load(f)
        for j in range(len(p['shapes'])):
            tmp_points = p['shapes'][j]['points']
            tmp_label = p['shapes'][j]['label']
            if is_all_chinese(tmp_label):
                if len(tmp_label) == 1:
                    save_label = tmp_label
                    save_points = tmp_points
                    img_label.append(tmp_label)
                    img_points.append(tmp_points)
        all_names.append(p['imagePath'])
        all_label.append(list(img_label))
        all_points.append(list(img_points))
        all_width.append(p['imageWidth'])
        all_height.append(p['imageHeight'])   

In [47]:
count = 0 
###[image_index][label_index][label_0, 2][x, y]
for i in tqdm(range(len(all_names))):
    for j in range(len(all_label[i])):
        pure_name = all_names[i].split('.')[0]
        
        Xmin = all_points[i][j][0][0]
        Ymin = all_points[i][j][0][1]
        Xmax = all_points[i][j][2][0]
        Ymax = all_points[i][j][2][1]
        text = all_label[i][j]
        
        if Xmin < Xmax and Ymin < Ymax and Xmin > 0 and Ymin >0 and Xmax >0 and Ymax >0:
            tmp_img = cv2.imread(os.path.join(train_path, 'img', all_names[i]))
            tmp_img = tmp_img[Ymin:Ymax, Xmin:Xmax]
            
            count += 1
            cv2.imwrite(os.path.join(train_path, 'only_text', pure_name + '_' + str(j) + '.jpg'), tmp_img)
            with open(os.path.join(train_path, 'merge_training', 'text_label.csv'), 'a+', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([os.path.join(train_path, 'only_text', pure_name + '_' + str(j) + '.jpg'), all_label[i][j]])
           

In [48]:
if not os.path.exists(os.path.join(train_path, 'txt')):
    os.mkdir(os.path.join(train_path, 'txt'))
    print('make direction')

make direction


In [49]:
###[image_index][label_index][label_0, 2][x, y]
for i in tqdm(range(len(all_names))):
    for j in range(len(all_label[i])):
    
        Xmin = all_points[i][j][0][0]
        Ymin = all_points[i][j][0][1]
        Xmax = all_points[i][j][2][0]
        Ymax = all_points[i][j][2][1]

        x = ((Xmin+Xmax) / 2) / int(all_width[i])
        y = ((Ymin+Ymax) / 2) / int(all_height[i])
        w = (Xmax-Xmin) / int(all_width[i])
        h = (Ymax-Ymin) / int(all_height[i])
        
        with open(os.path.join(train_path, 'txt', str(all_names[i].split('.')[0])) + '.txt', 'a+', newline='\n') as csvfile:
            writer = csv.writer(csvfile, delimiter = '\t')
            writer.writerow(['0', str(x), str(y), str(w), str(h)])

In [51]:
all_list = natsort(os.listdir(os.path.join(train_path, 'txt')))

In [52]:
random.seed(1)
shuffle(all_list)
train_data = all_list[:int(len(all_list) * 0.8)]
val_data = all_list[int(len(all_list) * 0.8):]

In [53]:
if not os.path.exists(os.path.join(train_path, 'split')):
    os.mkdir(os.path.join(train_path, 'split'))
    print('make direction')
if not os.path.exists(os.path.join(train_path, 'split','train')):
    os.mkdir(os.path.join(train_path, 'split','train'))
    print('make direction')
if not os.path.exists(os.path.join(train_path, 'split','val')):
    os.mkdir(os.path.join(train_path, 'split','val'))
    print('make direction')

make direction
make direction
make direction


In [55]:

for i in tqdm(range(len(train_data))):
    pure_name = train_data[i].split('.')[0]
    shutil.copy(os.path.join(train_path, 'txt', pure_name + '.txt'), os.path.join(train_path, 'split', 'train', pure_name + '.txt'))
    shutil.copy(os.path.join(train_path, 'img', pure_name + '.jpg'), os.path.join(train_path, 'split', 'train', pure_name + '.jpg'))

for i in tqdm(range(len(val_data))):
    pure_name = val_data[i].split('.')[0]
    shutil.copy(os.path.join(train_path, 'txt', pure_name + '.txt'), os.path.join(train_path, 'split', 'val', pure_name + '.txt'))
    shutil.copy(os.path.join(train_path, 'img', pure_name + '.jpg'), os.path.join(train_path, 'split', 'val', pure_name + '.jpg'))    

In [56]:
glob_train = natsort(glob.glob(os.path.join(train_path, 'split', 'train', '*.jpg')))
glob_val = natsort(glob.glob(os.path.join(train_path, 'split', 'val', '*.jpg')))

In [57]:
with open(os.path.join(darknet_path, 'data', 'train.txt'), 'w', newline='\n') as csvfile:
    writer = csv.writer(csvfile)
    for i in range(len(glob_train)):
        writer.writerow([glob_train[i]])

In [58]:
with open(os.path.join(darknet_path, 'data', 'val.txt'), 'w', newline='\n') as csvfile:
    writer = csv.writer(csvfile)
    for i in range(len(glob_val)):
        writer.writerow([glob_val[i]])

In [59]:
test_list = natsort(glob.glob(os.path.join(test_path, '*.jpg')))

In [60]:
with open(os.path.join(darknet_path, 'data', 'test.txt'), 'w', newline='\n', encoding='utf-8') as f:
#     writer = csv.writer(csvfile)
    for i in range(len(test_list)):
#         writer.writerow([test_list[i]])
        f.write(test_list[i] + '\n')

## Merge CSV

In [65]:
df1 = pd.read_csv(os.path.join(train_path, 'merge_training', 'text_label.csv'), header = None)
df2 = pd.read_csv(os.path.join(train_path, 'merge_training', 'hand_write_text_label_all.csv'), header = None)

In [66]:
df1_name_list = list(df1[int(len(df1) * 0.2):][0])
df1_label_list = list(df1[int(len(df1) * 0.2):][1])
df2_name_list = list(df2[0])
df2_label_list = list(df2[1])

In [67]:
Ser1 = pd.Series(df1_name_list + df2_name_list)
Ser2 = pd.Series(df1_label_list + df2_label_list)

train_dataset = pd.DataFrame({'0':Ser1, '1':Ser2})
val_dataset = df1[:int(len(df1) * 0.2)]

In [69]:
train_dataset.to_csv(os.path.join(train_path, 'merge_training', 'merge_training_all_handwrite.csv'), index=False)
val_dataset.to_csv(os.path.join(train_path, 'merge_training', 'merge_validation_all_handwrite.csv'), index=False)